# Propositional logic with exhaustification (ExhPL)

In [1]:
{-# LANGUAGE OverloadedStrings, FlexibleInstances #-}

import Prelude hiding (lookup)

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings, FlexibleInstances #-}
Why not:
{-# LANGUAGE FlexibleInstances #-}

## Syntax

We start by extending the datatype for expressions of Propositional Logic from the `hatt` library.

### Variables and wffs

Variables are simply characters. Variables with the same name will always be assigned the same truth-value by a given assignment. Following the `hatt` library, we define a recursive datatype for wffs of exhaustified propositional logic. The only addition here is the `Exh` data constructor.

In [2]:
newtype Var = Var Char
    deriving (Eq, Ord)
    
data Expr = Variable Var
          | Negation Expr
          | Exh Expr
          | ExhIE Expr
          | Conjunction Expr Expr
          | Disjunction Expr Expr
          deriving Eq

instance Show Var where
    show (Var v) = [v]

### Readable wffs

In [3]:
showBinaryConnective :: (Expr -> String) -> String -> Expr -> Expr -> String
showBinaryConnective show_ symbol exp1 exp2 =
  '(' : show_ exp1 ++ " " ++ symbol ++ " " ++ show_ exp2 ++ ")"

showBC :: String -> Expr -> Expr -> String
showBC = showBinaryConnective show
          
instance Show Expr where
  show (Variable      name)      = show name
  show (Negation      expr)      = '~' : show expr
  show (Exh expr) = "exh " ++ show expr
  show (ExhIE expr) = "exh-ie " ++ show expr
  show (Conjunction   exp1 exp2) = showBC "&&" exp1 exp2
  show (Disjunction   exp1 exp2) = showBC "||" exp1 exp2

### Wffs rendered in `LaTeX`

In [4]:
import Text.LaTeX
import Text.LaTeX.Packages.AMSMath (wedge, vee)
import Text.LaTeX.Base.Class

neg :: LaTeXC l => l
neg = comm0 "neg"

toLaTeX :: Expr -> LaTeX
toLaTeX (Variable (Var v)) = fromString [v]
toLaTeX (Negation expr) = autoParens $ neg <> toLaTeX expr
toLaTeX (Exh expr) = autoParens $ "exh" <> toLaTeX expr
toLaTeX (ExhIE expr) = autoParens $ "exh-ie" <> toLaTeX expr
toLaTeX (Conjunction expr1 expr2) = autoParens $ toLaTeX expr1 `wedge` toLaTeX expr2
toLaTeX (Disjunction expr1 expr2) = autoParens $ toLaTeX expr1 `vee` toLaTeX expr2

import IHaskell.Display

instance IHaskellDisplay Expr where
  display = display . math . toLaTeX

### TODO Parsing expressions with `parsec`

### Computing Structural Alternatives

In [99]:
binaryConnectives :: [Expr -> Expr -> Expr]
binaryConnectives = [Conjunction,Disjunction]

class Alt a where
    alt :: a -> [a]

instance Alt Expr where
    alt (Negation exp) = alt exp ++ map Negation (alt exp)
    alt (Exh exp) = alt exp ++ map Exh (alt exp)
    alt (ExhIE exp) = alt exp ++ map ExhIE (alt exp)
    alt (Conjunction exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- alt exp1, exp2' <- alt exp2] ++ alt exp1 ++ alt exp2
    alt (Disjunction exp1 exp2) = [f' exp1' exp2' | f' <- binaryConnectives, exp1' <- alt exp1, exp2' <- alt exp2] ++ alt exp1 ++ alt exp2
    alt exp = [exp]

In [100]:
alt $ Variable (Var 'a') `Disjunction` Variable (Var 'b') `Disjunction` Variable (Var 'c')

## Semantics

In [101]:
import Data.Map (Map, fromList, lookup)
import Control.Monad (liftM, liftM2, replicateM, filterM)
import Data.Maybe (fromMaybe)
import Data.List (group, sort, intersperse, intersect, maximumBy, nub)
import Data.Function (on)

Formulae are interpreted relative to assignments of variables to truth-values.

## Innocent exclusion

### Helper functions

In [112]:
subsets :: [a] -> [[a]]
subsets = filterM (const [True, False])

isSubset :: Eq a => [a] -> [a] -> Bool
isSubset a b = all (`elem` b) a

isMaximum :: Eq a => [a] -> [[a]] -> Bool
isMaximum x xs = not $ any (\x' -> isSubset x x' && x /= x') xs && x `elem` xs

toSet :: [LaTeX] -> LaTeX
toSet = autoBraces . mconcat . intersperse ","

displayAsSets :: [[Expr]] -> [LaTeX]
displayAsSets = map ((math . toSet) . map toLaTeX)

removeNonMaximal :: Eq a => [[a]] -> [[a]]
removeNonMaximal xs = filter (`isMaximum` xs) xs

In [114]:
type Mapping = Map Var Bool

variables :: Expr -> [Var]
variables expr = let vars_ (Variable      v)     vs = v : vs
                     vars_ (Negation      e)     vs = vars_ e vs
                     vars_ (Exh e) vs = vars_ e vs
                     vars_ (ExhIE e) vs = vars_ e vs
                     vars_ (Conjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                     vars_ (Disjunction   e1 e2) vs = vars_ e1 vs ++ vars_ e2 vs
                 in  map head . group . sort $ vars_ expr []
                 
assignments :: Expr -> [Mapping]
assignments expr = let vs = variables expr
                       ps = replicateM (length vs) [True, False]
                   in  map (fromList . zip vs) ps
                   
-- | Determines whether or not an alternative is excludable
excl :: Expr -> Expr -> Bool
excl s alt = not . isContradiction $ s `Conjunction` Negation alt

-- | Returns the list of excludable alternatives from a list of alternatives
exclAlts :: Expr -> [Expr]
exclAlts expr = filter (excl expr) (alt expr)

exclAltsIE :: Expr -> [Expr]
exclAltsIE = foldr1 intersect . removeNonMaximal . consistentSets

implicate :: Expr -> [Expr] -> Mapping -> Bool
implicate expr alts vs = foldr ((&&) . not . (`interpret` vs)) (interpret expr vs) alts

interpret :: Expr -> Mapping -> Bool
interpret (Variable      v)         vs = fromMaybe False (lookup v vs)
interpret (Negation      expr)      vs = not $ interpret expr vs
interpret (Exh expr) vs =  implicate expr (exclAlts expr) vs
interpret (ExhIE expr) vs = implicate expr (exclAltsIE expr) vs
interpret (Conjunction   exp1 exp2) vs = interpret exp1 vs && interpret exp2 vs
interpret (Disjunction   exp1 exp2) vs = interpret exp1 vs || interpret exp2 vs
                                    

-- | Determines whether an expression is contradictory.
isContradiction :: Expr -> Bool
isContradiction = not . or . values

values :: Expr -> [Bool]
values expr = map (interpret expr) (assignments expr)

-- I can just do this in the semantics
isConsistent :: Expr -> [Expr] -> Bool
isConsistent expr alts = (not . isContradiction) $ foldr (Conjunction . Negation) expr alts

consistentSets :: Expr -> [[Expr]]
consistentSets expr = filter (isConsistent expr) (subsets (exclAlts expr))

In [106]:
values (Exh (Variable (Var 'a') `Disjunction` Variable (Var 'b')))

[False,False,False,False]

In [107]:
values (ExhIE (Variable (Var 'a') `Disjunction` Variable (Var 'b')))

[False,True,True,False]

In [108]:
equivalent :: Expr -> Expr -> Bool
equivalent exp1 exp2 = values exp1 == values exp2

# TODO Truth tables (preferably in LateX)

In [109]:
aOrB :: Expr
aOrB = Exh (Variable (Var 'a') `Disjunction` Variable (Var 'b'))

In [110]:
variables aOrB

[a,b]

In [111]:
assignments aOrB

[fromList [(a,True),(b,True)],fromList [(a,True),(b,False)],fromList [(a,False),(b,True)],fromList [(a,False),(b,False)]]